# Загрузка данных / Data loading
Датасет из соревнования https://www.kaggle.com/c/nsu-abmd-2021-e-commerce#. Необходимо предсказать уровень продаж Sales

In [ ]:
import pandas as pd

df = pd.read_csv('data/train.csv')

df

# Исследование данных / Data exploration
Data exploration is the first step of data analysis used to explore and visualize data to uncover insights from the start or identify areas or patterns to dig into more. Using interactive dashboards and point-and-click data exploration, users can better understand the bigger picture and get to insights faster.

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

In [ ]:
df.Rating = df.Rating.str.replace(',', '.').astype(float)
df.Category = df.Category.str[1:]

In [ ]:
df.isna().any()

## Количественные / Quantitative

In [ ]:
df.hist(figsize=(15, 10), bins=25);

In [ ]:
df[df.Sales>2000].Category.value_counts()

In [ ]:
df[df["Final price"]>1e6].shape

In [ ]:
df.boxplot(figsize=(20, 5), rot=45); #.filter(regex=("^((?!rice).)*$"))

In [ ]:
df[df.Comments>2_000] #Price

## Категориальные / Category

In [ ]:
df.select_dtypes('O')

In [ ]:
import numpy as np


pd.set_option('min_rows', 500)
np.set_printoptions(edgeitems=100)

In [ ]:
for i in df.select_dtypes('O').columns:
    print(i, df[i].unique())

In [ ]:
#df.Deliveryscheme.value_counts(dropna=False).plot.barh();

In [ ]:
df = pd.concat([df, df.full_category.str.split('/', expand=True)\
                .rename(columns={0: 'cat_meta', 1: 'cat_sub', 2: 'cat_sub_sub'})], axis=1)

df

In [ ]:
df.cat_meta.value_counts(dropna=False).plot.barh();

In [ ]:
df.boxplot(column='Sales', by='cat_meta', figsize=(5, 10), vert=False);

In [ ]:
df.cat_sub.value_counts(dropna=False)

In [ ]:
df.cat_sub.value_counts(dropna=False, ascending=True).plot.barh(figsize=(5, 50));

In [ ]:
df.cat_sub = df.cat_sub.astype('object')
df.cat_sub_sub = df.cat_sub_sub.astype('object')
df.Brand = df.Brand.astype('object')

In [ ]:
df.loc[df.groupby('cat_sub').Sales.transform('count')<1000, 'cat_sub'] = 'Rare'
df.loc[df.groupby('cat_sub_sub').Sales.transform('count')<1000, 'cat_sub_sub'] = 'Rare'

In [ ]:
df.groupby(['cat_sub'])['Sales'].max().sort_values(ascending=True).plot.barh(figsize=(5, 50));

In [ ]:
t = df.groupby(['cat_sub'])['Sales'].median().sort_values(ascending=True)
df['cat_sub'] = df['cat_sub'].astype(pd.CategoricalDtype(list(t.index), ordered=True))

df.boxplot(column='Sales', by='cat_sub', figsize=(10, 50), vert=False);

In [ ]:
df.cat_sub_sub.value_counts(dropna=False)

In [ ]:
df[df.groupby('cat_sub_sub').Sales.transform('count')>1].Sales.value_counts(bins=25)#.cat_sub_sub.value_counts(dropna=False)

In [ ]:
df[df.groupby('cat_sub_sub').Sales.transform('count')>500]\
    .boxplot(column='Sales', by='cat_sub_sub', figsize=(10, 50), vert=False);

In [ ]:
df.Seller.value_counts(dropna=False)

In [ ]:
df.Brand.value_counts(dropna=False)

In [ ]:
df[df.groupby('Brand').Sales.transform('count')>1].Sales.value_counts(bins=15)#.cat_sub_sub.value_counts(dropna=False)

In [ ]:
df.loc[df.groupby('Brand').Sales.transform('count')<50, 'Brand'] = 'Rare'

In [ ]:
t = df.groupby(['Brand'])['Sales'].median().sort_values()
df['Brand'] = df['Brand'].astype(pd.CategoricalDtype(list(t.index), ordered=True))

df.boxplot(column='Sales', by='Brand', figsize=(10, 100), vert=False);

# Чистка данных / Data cleaning
The main aim of Data Cleaning is to identify and remove errors & duplicate data, in order to create a reliable dataset. This improves the quality of the training data for analytics and enables accurate decision-making.

## Пустые значения / Missing values

In [ ]:
# for i in df.select_dtypes('number').columns:
#     print(i, df[i].unique())

In [ ]:
df.info()

In [ ]:
df_nan = df[['Name', 'Brand', 'Seller', 'Comments', 'Final price', 'Max price', 'Min price', 'Average price', 'Sales', 'Days in stock', 'Days with sales', 'Rating', 'Basic Sale', 'Basic Sale Price', 'Base price', 'cat_sub', 'cat_sub_sub']].copy()

df_nan.info()

In [ ]:
df_nan.info()

In [ ]:
df_nan.isna().any()

In [ ]:
df_nan[df_nan.isna().any(axis=1)]#.style.highlight_null()

In [ ]:
df_nan['Average price'].fillna((df_nan['Max price']+df_nan['Min price']) / df_nan.Sales, inplace=True)

In [ ]:
df_nan.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
df['Price diff'] = (df['Max price'] - df['Min price'])

In [ ]:
df.sort_values(by='Price diff', ascending=False)

In [ ]:
df_nan = df_nan.drop(columns=['Max price', 'Min price', 'Average price'])

## One-hoting

In [ ]:
df_one_hoted = df_nan.copy()

ONE_HOT_COL = ['Brand', 'cat_sub_sub', 'cat_sub'] 

temp = pd.get_dummies(df_one_hoted[ONE_HOT_COL], prefix=ONE_HOT_COL, dtype=bool) #, sparse=True

df_one_hoted = pd.concat([temp, df_one_hoted.select_dtypes('number'), 
                            df_one_hoted.select_dtypes('bool')], axis=1)

df_nan.shape, df_one_hoted.shape

In [ ]:
df_one_hoted.info()

# Создание модели / Model creation

In [ ]:
from sklearn.model_selection import train_test_split


df_sampled = df_one_hoted.sample(25_000)

X_train, X_test, y_train, y_test = train_test_split(df_sampled.drop(columns='Sales'), df_sampled.Sales)

In [ ]:
df_sampled.info()

In [ ]:
df.Sales.describe()

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


model = RandomForestRegressor(n_jobs=-1, max_depth=10)
model.fit(X_train, y_train)

model.feature_names = list(X_train.columns.values)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(f'MAE train: {mean_absolute_error(y_train, y_pred_train)}, MAE test: {mean_absolute_error(y_test, y_pred_test)}')
print(f'MSE train: {mean_squared_error(y_train, y_pred_train)}, MSE test: {mean_squared_error(y_test, y_pred_test)}')
print(f'MAPE train: {mean_absolute_percentage_error(y_train, y_pred_train)}, MAPE test: {mean_absolute_percentage_error(y_test, y_pred_test)}')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(model, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_train, y_train)
rs.best_params_

#{'n_estimators': 600,
# 'min_samples_split': 23,
# 'min_samples_leaf': 2,
# 'max_features': 'sqrt',
# 'max_depth': 15,
# 'bootstrap': False}

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
rs_df.head(10)

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [600,400,700]
max_features = ['sqrt']
max_depth = [15,14,14]
min_samples_split = [23,12]
min_samples_leaf = [2,7]
bootstrap = [False]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(model, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_train, y_train)

rfc_3 = gs.best_estimator_
gs.best_params_

In [ ]:
#{'n_estimators': 600,
# 'min_samples_split': 23,
# 'min_samples_leaf': 2,
# 'max_features': 'sqrt',
# 'max_depth': 15,
# 'bootstrap': False}

#{'bootstrap': False,
# 'max_depth': 15,
# 'max_features': 'sqrt',
# 'min_samples_leaf': 2,
# 'min_samples_split': 12,
# 'n_estimators': 400}

#model = RandomForestRegressor(n_estimators=400, min_samples_split=12, min_samples_leaf=2, max_features="sqrt", n_jobs=-1, max_depth=15, bootstrap=False)
#model.fit(X_train, y_train)
#
#model.feature_names = list(X_train.columns.values)
#
#y_pred_train = model.predict(X_train)
#y_pred_test = model.predict(X_test)
#
#print(f'MAE train: {mean_absolute_error(y_train, y_pred_train)}, MAE test: {mean_absolute_error(y_test, y_pred_test)}')
#print(f'MSE train: {mean_squared_error(y_train, y_pred_train)}, MSE test: {mean_squared_error(y_test, y_pred_test)}')
#print(f'MAPE train: {mean_absolute_percentage_error(y_train, y_pred_train)}, MAPE test: {mean_absolute_percentage_error(y_test, y_pred_test)}')

In [ ]:
list(zip(y_pred_train, y_train))

In [ ]:
(y_pred_train - y_train).value_counts(bins=25)#.hist(bins=100)

In [ ]:
from bokeh.palettes import Colorblind8

pd.Series(model.feature_importances_, index=X_test.columns)\
    .sort_values()[-25:].plot.barh(color=Colorblind8, figsize=(15, 5));

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance


result = permutation_importance(model, X_test, y_test, n_repeats=3, n_jobs=-1)

sorted_idx = result.importances_mean.argsort()

In [ ]:
plt.boxplot(result.importances[sorted_idx][-10:].T, vert=False, labels=X_test.columns[sorted_idx][-10:]);

In [ ]:
plt.boxplot(result.importances[sorted_idx][:10].T, vert=False, labels=X_test.columns[sorted_idx][:10]);

In [ ]:
from sklearn.feature_selection import RFE


rfe = RFE(model, n_features_to_select=15)
rfe.fit(X_test, y_test)

display(rfe.score(X_test, y_test))

pd.DataFrame({'support': rfe.support_, 'rank': rfe.ranking_}, index=X_test.columns).sort_values(by='rank')

In [ ]:
df_final = pd.read_csv('data/test.csv')

df_final.Rating = df_final.Rating.str.replace(',', '.').astype(float)
df_final.Category = df_final.Category.str[1:]

df_final = pd.concat([df_final, df_final.Category.str.split('/', expand=True)\
                .rename(columns={0: 'cat_meta', 1: 'cat_sub', 2: 'cat_sub_sub'})], axis=1)

df_final.cat_sub = df_final.cat_sub.astype('object')
df_final.cat_sub_sub = df_final.cat_sub_sub.astype('object')
df_final.Brand = df_final.Brand.astype('object')



df_final_nan = df_final[['Name', 'Seller', 'Brand', 'Comments', 'Final price', 'Max price', 'Min price', 'Average price', 
'Days in stock', 'Days with sales', 'Rating', 'Basic Sale', 'Basic Sale Price', 'Base price', 'cat_sub', 'cat_sub_sub']].copy()

df_final_nan[df_final_nan.isna().any(axis=1)]#.style.highlight_null()

df_final_nan.corr().style.background_gradient(cmap='coolwarm')

df_final['Price diff'] = (df_final['Max price'] - df_final['Min price'])

df_final.sort_values(by='Price diff', ascending=False)

df_final_nan = df_final_nan.drop(columns=['Max price', 'Min price', 'Average price'])

df_final_one_hoted = df_final_nan.copy()

ONE_HOT_COL = ['Seller', 'Brand', 'cat_sub_sub', 'cat_sub'] 

temp = pd.get_dummies(df_final_one_hoted[ONE_HOT_COL], prefix=ONE_HOT_COL, dtype=bool) #, sparse=True

df_final_one_hoted = pd.concat([temp, df_final_one_hoted.select_dtypes('number'), 
                            df_final_one_hoted.select_dtypes('bool')], axis=1)

In [ ]:
df_final_one_hoted["Brand_Rare"] = False
df_final_one_hoted["cat_sub_Rare"] = False

In [ ]:
df_final_one_hoted["cat_sub_sub_Rare"] = False
df_final_one_hoted["cat_sub_sub_Измельчение и смешивание"] = False
df_final_one_hoted["cat_sub_sub_Медицинские приборы"] = False
df_final_one_hoted["cat_sub_sub_Приготовление блюд"] = False
df_final_one_hoted["cat_sub_sub_Приготовление напитков"] = False
df_final_one_hoted["cat_sub_sub_Фены, приборы для укладки"] = False
df_final_one_hoted["cat_sub_Садовая техника"] = False
df_final_one_hoted["cat_sub_Красота и здоровье"] = False
df_final_one_hoted["cat_sub_Техника для дома"] = False
df_final_one_hoted["cat_sub_Техника для кухни"] = False

In [ ]:
scores = rfe.predict(df_final_one_hoted[model.feature_names])

In [ ]:
print("Id,Expected")
for i in range(len(scores)):
    print(i, scores[i], sep=',')